In [ ]:
# We need SageMaker SDK >1.50.12 for imx8qm DLR (https://github.com/aws/sagemaker-python-sdk/blob/master/CHANGELOG.md#v15012-2020-02-17)
!pip install --upgrade sagemaker --quiet

In [ ]:
import sagemaker

session = sagemaker.session.Session()
default_s3_bucket = 's3://{}'.format(session.default_bucket())
print('default_s3_bucket: {}'.format(default_s3_bucket))

In [ ]:
role = sagemaker.get_execution_role()
print("Using IAM role arn: {}".format(role))

# create a descriptive job name 
job_name_prefix = 'pasta-ssd-voc'
print("job_name_prefix: {}".format(job_name_prefix))

s3_output_path = '{}/{}/output'.format(default_s3_bucket,job_name_prefix)
print("S3 model output path: {}".format(s3_output_path))

training_images = session.upload_data('./VOC2019', key_prefix='{}/dataset/VOC2019'.format(job_name_prefix))
print("Training images S3 Uri: {}".format(training_images))

In [ ]:
training_images = session.upload_data('./dataset', key_prefix='{}/dataset'.format(job_name_prefix))
print("Training images S3 Uri: {}".format(training_images))

In [ ]:
static_hyperparameters = {
    'epochs': 2,
    'num-workers': 12,
    'batch-size': 10,
    'network': 'mobilenet1.0',
    'data-shape': 512
}
print("static_hyperparameters: {}".format(static_hyperparameters))

instance_type = "ml.p3.2xlarge"

In [ ]:
from sagemaker.mxnet.estimator import MXNet
estimator = MXNet(entry_point="train_ssd_denis.py",
                  role=role,
                  train_instance_type=instance_type,
                  train_instance_count=1,
                  output_path=s3_output_path,
                  framework_version="1.4.1",
                  py_version='py3',
                  base_job_name=job_name_prefix,
                  hyperparameters=static_hyperparameters
                 )

In [ ]:
estimator.fit({
    "train": training_images
})

In [ ]:
compiled_output_path = '{}/{}/output'.format(default_s3_bucket,job_name_prefix)

optimized_ic = estimator.compile_model(
    target_instance_family='imx8qm', 
    input_shape={'data':[1, 3, 224, 224]},  # Batch size 1, 3 channels, 224x224 Images.
    output_path=compiled_output_path,
    role=role,
    framework='mxnet', framework_version='1.2.1')